# Profiling and Optimization Hands-On

In [ ]:
import numpy as np
from math import sin, cos
import matplotlib.pyplot as plt
from numba import njit
%matplotlib inline

## A common problem: Which implementation is faster?

Example: Given a large 2D array, we will explore different ways to create the array and to calculate its mean. Determine which one is fastest, using the `%timeit` notebook function.

### setup: define our 2D array:

We'll make some dummy test data that looks like:

$M_{ij} = \sin(i)\cos(0.1 j)$

and we will construct this array in multiple ways.

In [ ]:
def create_array_loop(N,M):
    arr = []
    for y in range(M):
        row = []
        for x in range(N):
            row.append(sin(x)*cos(0.1*y))
        arr.append(row)
    return arr

def create_array_list(N,M):
    """a 2D array using a list-comprehension"""
    return [[sin(x)*cos(0.1*y) for x in range(N)] for y in range(M)]  

def create_array_np(N,M):
    """ a 2D array using numpy"""
    X,Y = np.meshgrid(np.arange(N), np.arange(M))
    return np.sin(X)*np.cos(0.1*Y)



Let's first just plot the arrays, to see if they are the same:

In [ ]:
N=10; M=30  # our array dimensions
plt.figure(figsize=(12,5))
plt.subplot(1,3,1)
plt.imshow( create_array_loop(N,M))
plt.subplot(1,3,2)
plt.imshow( create_array_list(N,M))
plt.subplot(1,3,3)
plt.imshow( create_array_np(N,M))

### Task: determine which array creation routine is fastest

And make a plot of the speed of each!   Does the result change much when the array size becomes larger? Try much larger sizes for N and M

Hint: use the `%timeit -o` magic function to have `%timeit` return results (see the timeit help)

### SOLUTION:

Note: to minimize the uncertainty, try increasing the problem size! 

In [ ]:
N=10; M=30

**Useful tip**: timeit has a *-o* option that lets you save the results for later comparison!

In [ ]:
t = {}  # a place to record out time statistics

In [ ]:
t['loop'] = %timeit -o create_array_loop(N,M)

In [ ]:
t['list'] = %timeit -o create_array_list(N,M)

In [ ]:
t['numpy']  = %timeit -o create_array_np(N,M)

In [ ]:
def plot_performance(time_dict):
    mean = [t.average for t in time_dict.values()]
    std = [t.stdev for t in time_dict.values()]
    x = range(len(time_dict))
    plt.errorbar(x, mean, yerr=std, lw=3, fmt="o") 
    plt.xticks( np.arange(len(time_dict)), time_dict.keys())
    plt.ylabel("average execution time (s)")
    plt.grid()

In [ ]:
plot_performance(t)

## Task 2: determine the fastest way to find the mean of our array

note that `create_array_list()` and `create_array_loop` both return a list-of-lists, while `create_array_np` returns a 2D numpy array.   There are multiple ways to compute the mean of these arrays.  See again which is fastest!

try at least:

1. using the built-in python `sum` function and either a for-loop or list-comprehension
2. using pure numpy (e.g. `array.mean()`)
3. other ways you can think of!

### SOLUTION

In [ ]:
N=100; M=100
a_list = create_array_list(N,M)
a_np = create_array_np(N,M)

In [ ]:
sum([sum(x) for x in a_list])/(N*M)

In [ ]:
a_np.mean()

In [ ]:
t2 = {}
t2["numpy"] = %timeit -o a_np.mean()

In [ ]:
t2["sum"] = %timeit -o sum([sum(x) for x in a_list])/(N*M)

In [ ]:
plot_performance(t2)